# API Code File

네이버
import os
import sys
import urllib.request
import json
import pandas as pd
def getresult(client_id,client_secret,query,display=10,start=1,sort='sim'):
    encText = urllib.parse.quote(query)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + \  
    "&display=" + str(display) + "&start=" + str(start) + "&sort=" + sort
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText \
    # "&display=" + str(display) + "&start=" + str(start) + "&sort=" + sort  xml 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        response_json = json.loads(response_body)
    else:
        print("Error Code:" + rescode)

    return pd.DataFrame(response_json['items'])

client_id = "IfaUFMdpnoeS4Kdtzb1R"
client_secret = "eAdzXD8iBG"
query = '홍대입구역 맛집'
display=100
start=1
sort='sim'

result_all=pd.DataFrame()
for i in range(0,2):
    start= 1 + 100*i
    result= getresult(client_id,client_secret,query,display,start,sort)
    
    result_all=pd.concat([result_all,result])

공공 데이터포털 API 활용 크롤링
import requests
import pandas as pd

# 발급받은 API 키
api_key = '2CGGfCCxYUKw%2FCLgS2z26RYvp%2BIFowLHL24iCF8hDVThaIbj7MC0C0dTGqEXqPgmuhRyg88uoGWyVnYkbcYZfg%3D%3D'

# API 요청 URL 기본 템플릿 (API 문서에서 확인한 URL로 변경해야 합니다)
base_url = 'https://api.odcloud.kr/api/15060386/v1/uddi:9951ec3f-d1c9-49e8-9ed4-f026c39a7925'

# 요청할 페이지 수와 한 페이지당 행 수
total_pages = 100  # 요청할 페이지 수 (필요에 따라 조절)
rows_per_page = 1000  # 한 페이지당 데이터 수

# 빈 리스트 생성 (모든 데이터를 저장할 리스트)
all_data = []

# 각 페이지에 대해 반복하여 요청
for page in range(1, total_pages + 1):
    # 요청 URL 생성 (API 문서에서 제공하는 형식으로 조정해야 합니다)
    url = f'{base_url}?serviceKey={api_key}&page={page}&perPage={rows_per_page}'
    
    # API 요청 보내기
    response = requests.get(url)
    
    # 응답 확인
    if response.status_code == 200:
        # JSON 파싱
        json_to_dict = response.json()
        
        # 데이터가 'data'에 있는 경우
        items = json_to_dict.get('data', [])
        
        # 데이터를 all_data 리스트에 추가
        all_data.extend(items)
    else:
        print(f"Error: {response.status_code} on page {page}")

# DataFrame으로 변환
df_2023 = pd.DataFrame(all_data)

# 출력
print(df_2023)

API 공공데이터 여러개 한번에 
import requests
import pandas as pd

api_key = '2CGGfCCxYUKw%2FCLgS2z26RYvp%2BIFowLHL24iCF8hDVThaIbj7MC0C0dTGqEXqPgmuhRyg88uoGWyVnYkbcYZfg%3D%3D'

# 기본 URL 및 엔드포인트 리스트
base_url = 'https://api.odcloud.kr/api/15060386/'

url_list = [
    'v1/uddi:9951ec3f-d1c9-49e8-9ed4-f026c39a7925',
    'v1/uddi:cb73d6d5-064c-4dd2-a136-8c3069aa1fe2',
    'v1/uddi:dd407ff5-f23a-4d46-b90b-dc37505fb02c',
    'v1/uddi:bd8a7575-d4c9-4a22-a972-fac12348dd7e',
    'v1/uddi:65991a70-9fcf-40c3-ad3a-ee24b401c56a',
    'v1/uddi:08f09298-4bae-41a7-a315-c1421a1d418b',
    'v1/uddi:52bbace2-f0c1-46c9-9057-c31731da9b30',
    'v1/uddi:d832fa12-7b66-4058-beae-e23267270a05',
    'v1/uddi:1e3d031d-8650-45db-9daf-bb742cfbb83c'
]

total_pages_list = [39, 41, 37, 39, 41, 43, 45, 44, 45]
rows_per_page = 1000  # 한 페이지당 데이터 수

# 시작 연도 지정 (2023년부터 시작)
start_year = 2023

# URL 리스트와 페이지 리스트를 이중 for문으로 순회
for endpoint, total_pages in zip(url_list, total_pages_list):
    all_data = []  # 각 연도별 데이터를 저장할 리스트
    
    for page in range(1, total_pages + 1):
        # 요청 URL 생성
        request_url = f'{base_url}{endpoint}?serviceKey={api_key}&page={page}&perPage={rows_per_page}'
        
        # API 요청 보내기
        response = requests.get(request_url)
        
        # 응답 확인
        if response.status_code == 200:
            # JSON 파싱
            json_to_dict = response.json()
            
            # 데이터가 'data'에 있는 경우
            items = json_to_dict.get('data', [])
            
            # 데이터를 all_data 리스트에 추가
            all_data.extend(items)
        else:
            print(f"Error: {response.status_code} on page {page} for URL {endpoint}")
    
    # DataFrame으로 변환
    df = pd.DataFrame(all_data)
    
    # 동적으로 테이블명 생성 및 할당
    table_name = f'dt_{start_year}'
    globals()[table_name] = df  # globals()를 사용해 변수 생성
    
    # CSV 파일로 저장
    df.to_csv(f'{table_name}.csv', index=False, encoding='ms949')
    
    # 다음 연도로 이동
    start_year -= 1

# 생성된 DataFrame 출력 예시 (원하는 경우)
print(dt_2023.head())